In [ ]:
import pandas as pd, numpy as np, xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON
import scipy.io

In [ ]:
import plotly
plotly.offline.init_notebook_mode()
plotly_config = {'scrollZoom': True, 'displaylogo': False, 'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': None,
    'width': None,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },
  'modeBarButtonsToAdd': 
    ['drawline',
    'drawopenpath',
    'drawclosedpath',
    'drawcircle',
    'drawrect',
    'eraseshape'
    ]
  
  }

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
config_path = Path(params["config_path"])
config = config_adapter.load(config_path)
RenderJSON(config)

In [ ]:
fs, data = scipy.io.wavfile.read(params["audio_path"])
song = xr.Dataset()
song["data"] = xr.DataArray(data, dims="t")
song["t"] = np.arange(data.size)/fs
song["t"].attrs["fs"] = fs
song

In [ ]:
if "data_selection" in config:
    song = song.sel(t=slice(config["data_selection"]["min_t"], config["data_selection"]["max_t"]))
song

In [ ]:
volume_params = config["volume"]
win_size = int(np.round(volume_params["window_duration"]*fs))
stride = int(np.round(fs/volume_params["approx_out_fs"]))
if volume_params["window_type"] == "hanning":
    window = xr.DataArray(np.hanning(win_size), dims="window")
else:
    raise Exception(f'Unhandled windowtype {volume_params["window_type"]}')
tmp = song["data"].rolling(t=win_size, center=True).construct("window", stride = stride).dropna(dim="t", how="any")
volume = xr.Dataset()
volume["volume"] =  np.log10(np.abs(tmp * window).mean("window"))
volume_fs = fs/stride
volume["t"].attrs["fs"] = volume_fs
volume

In [ ]:
win_size = config["spectrogram"]["nfft"]
stride = config["spectrogram"]["hop"]
spectro_window = song["data"].rolling(t=win_size, min_periods=win_size, center=True).construct("window_t", stride=stride)
if config["spectrogram"]["window_type"] =="hanning":
    spectro_window = spectro_window * xr.DataArray(np.hanning(spectro_window.sizes["window_t"]), dims="window_t")
else: raise Exception(f'Unknown window type {config["spectrogram"]["window_type"]}')
fft = xr.apply_ufunc(np.fft.rfft, spectro_window, input_core_dims=[["window_t"]], output_core_dims=[["f"]])
fft["f"] = np.fft.rfftfreq(spectro_window.sizes["window_t"], 1/fs)
fft = fft.sel(f=slice(config["spectrogram"]["f_bounds"][0], config["spectrogram"]["f_bounds"][1]))
psd = np.abs(fft)**2
display_psd = np.log10(psd)
display_psd

In [ ]:
if "annotation_path" in params:
    annotations = pd.read_csv(params["annotation_path"]).rename(columns={"name": "label", "start_seconds": "start", "stop_seconds": "end"}).sort_values("start")
else:
    annotations = pd.DataFrame([], columns=["label", "start", "end"])
annotations

In [ ]:
if "data_selection" in config:
    annotations = annotations[(annotations["start"] > config["data_selection"]["min_t"]) & (annotations["end"] < config["data_selection"]["max_t"])]
annotations

In [ ]:
all_time_series = xr.Dataset()
max = display_psd.max().item()
all_time_series["display_psd"] = display_psd.assign_attrs(fig_zmin=max/2, fig_zmax=max)
additional_time_series = dict(volume=volume["volume"])
if "additional_time_series_paths" in params:
    for p, key in params["additional_time_series_paths"]:
        d = xr.load_dataset(p)
        additional_time_series[key] = d[key]
for a in additional_time_series:
    all_time_series[a] = additional_time_series[a].interp(t=all_time_series["t"])
all_time_series
    

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
heights = [all_time_series[k].ndim for k in all_time_series.data_vars]
heights = heights/np.sum(heights)
fig = make_subplots(rows=len(all_time_series.data_vars), cols=1, row_heights=list(heights), shared_xaxes=True)
for i, (var, a) in enumerate(all_time_series.data_vars.items()):
    if a.ndim==1:
      fig.add_trace(go.Scatter(x=a["t"].values, y=a.values,
                      name=var,showlegend=False, **{k[4:]:v for k,v in a.attrs.items() if k.startswith("fig_")})
        ,row=i+1, col=1
      )
    elif a.ndim==2:
      other_dim = [d for d in a.dims if not d=="t"][0]
      fig.add_trace(go.Heatmap(x=a["t"].values, y=a[other_dim].values, z= a.transpose(other_dim, "t").values,
                     name=var,showlegend=False, colorbar=dict(len=heights[i]), colorbar_yanchor="middle",
                      colorbar_y=((np.sum(heights[i+1:]) + heights[i]/2)-0.5)*1.13+0.5,
                     **{k[4:]:v for k,v in a.attrs.items() if k.startswith("fig_")})
        ,row=i+1, col=1
      )
      ny =  len(np.unique(a[other_dim].values)) 
      if ny< 20:
         fig.update_yaxes(nticks=ny, row=i+1, col=1)
         
if "syb_annotations" in params:
    annot_table = annotations.assign(t=(annotations["start"]+annotations["end"])/2)[["t"]+params["syb_annotations"]]
    hover_template = ''.join([f'<br>\t{p}: '+ '%{customdata[' + str(i) + ']}' for i, p in enumerate(params["syb_annotations"])])
    fig.add_trace(go.Scatter(
        x=annot_table["t"],
        y=[display_psd["f"].mean().item()] * len(annot_table.index) ,
        customdata= annot_table[params["syb_annotations"]],
        mode='lines',
        opacity=0,
        hovertemplate =hover_template,
        showlegend=False,
        name="syb_info"
), row=1, col=1)
    
if not "other_bounds" in params:
    params["other_bounds"] = []
for row in annotations.to_dict(orient="index").values():
    fig.add_vrect(x0=row["start"], x1=row["end"], 
                label = dict(
                    text=row["label"],
                    textposition="top center",
                    font=dict(size=20, family="Times New Roman", color="white"),
                ),
                line=dict(color="MediumPurple"))
    for d in params["other_bounds"]:
        fig.add_vrect(x0=row[d["start"]], x1=row[d["end"]], 
                line=dict(color="yellow", dash="dot"))
    
# fig.update_traces(xaxis='x')
# fig.update_shapes(selector=dict(type="rect"), xref="x")

fig.update_layout(hovermode='x unified', hoversubplots="axis", xaxis_showticklabels=True)
fig.show(config = plotly_config)